# Taller 3

Considere un puente que se va deteriorando con el paso del tiempo. Asuma que el estado del puente se puede
clasificar en 4 estados {1, 2, 3, 4} donde 1 indica que el puente está en óptimas condiciones y 4 en muy mal estado.
Para intervenir el puente se consideran 3 acciones {1, 2, 3} donde 1 es un mantenimiento de rutina y 3 es un arreglo
estructural profundo. La dinámica del sistema es entonces la siguiente:

![title](Trans.png)

Finalmente, asuma que los costos de las intervenciones son independientes del estado del puente y son de 1, 4, 10
respectivamente.

## Punto 1

Para verificar que el modelo es unitario construyamos las matrices de transición asociadas a cada política estacionaria determinista posible. De estas hay $3^4$ posibles. Como son pocas, podemos verificar que todas las matrices de transición poseen solo una clase recurrente.

In [1]:
const P1=[0.1 0.6 0.2 0.1;0.8 0.2 0.0 0.0; 0.95 0.05 0.0 0.0]
const P2=[0.0 0.1 0.6 0.3;0.7 0.2 0.1 0.0; 0.85 0.1 0.05 0.0]
const P3=[0.0 0.0 0.2 0.8;0.3 0.4 0.2 0.1; 0.65 0.2 0.1 0.05]
const P4=[0.0 0.0 0.0 1.0;0 0.6 0.2 0.2; 0.5 0.5 0.0 0.0]

function p(j,s,a)
    if s==1
        return P1[a,j]
    elseif s==2
        return P2[a,j]
    elseif s==3
        return P3[a,j]
    elseif s==4
        return P4[a,j]
    else
        return 0.0
    end
end    

p (generic function with 1 method)

In [2]:
function construir_matriz(a,S)
    A=zeros(length(S),length(S))
    for i in 1:length(S)
        for j in 1:length(S)
            A[j,i]=p(S[i],S[j],a[j])
        end
    end
    return A
end

construir_matriz (generic function with 1 method)

In [3]:
using DiscreteMarkovChains

Veamos ahora cuales son las clases recurrentes de todas las posibles políticas estacionarias. La función de la libreria evalua la recurrencia de las clases en el vector booleano. Nunca hay más de un true, por lo que nunca hay más de una clase recurrente, esto es, el modelo es unichain.

In [4]:
matrices=[]
politicas=[]
for (i,a) in enumerate(Iterators.product(1:3, 1:3,1:3,1:3))
    mat=construir_matriz(a,1:4)
    append!(matrices,[mat])
    append!(politicas,[a])
    chain = DiscreteMarkovChain(mat)  
    println(i," politica: ",a," clases: ",periodicities(chain))
end

1 politica: (1, 1, 1, 1) clases: ([[4], [3], [2], [1]], Any[true, false, false, false], Any[1, 1, 1, 1])
2 politica: (2, 1, 1, 1) clases: ([[4], [3], [2], [1]], Any[true, false, false, false], Any[1, 1, 1, 1])
3 politica: (3, 1, 1, 1) clases: ([[4], [3], [2], [1]], Any[true, false, false, false], Any[1, 1, 1, 1])
4 politica: (1, 2, 1, 1) clases: ([[4], [3], [1, 2]], Any[true, false, false], Any[1, 1, 1])
5 politica: (2, 2, 1, 1) clases: ([[4], [3], [1, 2]], Any[true, false, false], Any[1, 1, 1])
6 politica: (3, 2, 1, 1) clases: ([[4], [3], [1, 2]], Any[true, false, false], Any[1, 1, 1])
7 politica: (1, 3, 1, 1) clases: ([[4], [3], [1, 2]], Any[true, false, false], Any[1, 1, 1])
8 politica: (2, 3, 1, 1) clases: ([[4], [3], [1, 2]], Any[true, false, false], Any[1, 1, 1])
9 politica: (3, 3, 1, 1) clases: ([[4], [3], [1, 2]], Any[true, false, false], Any[1, 1, 1])
10 politica: (1, 1, 2, 1) clases: ([[4], [1, 2, 3]], Any[true, false], Any[1, 1])
11 politica: (2, 1, 2, 1) clases: ([[4], [1, 

Para evitar que el modelo fuera unichain sería suficiente considerar un estado adicional de "puente mal construido", el cual solo puede ocurrir si fue mal construido en un principio y la unica acción posible para este estado es "nada". Así, partiendo de los estados normales no se puede llegar a estado y de este estado no se puede llegar a los anteriores. Esto haría que se tuvieran dos clases recurrentes.

## Punto 2

Definamos el ingreso $p(s)$ por mantener el puente en estado $s$ como
\begin{equation}
    p(s)=\begin{cases}
        10 & \text{if } s=0\\
        5 & \text{if } s=1\\
        -1 & \text{if } s=2\\
        -10 & \text{if } s=3\\
    \end{cases}
\end{equation}

In [5]:
function p_rec(s)
    if s==1
        return 10.0
    elseif s==2
        return 5.0
    elseif s==3
        return -1.0
    else s==4
        return -10.0
    end
end

function r1(sig,s,a)
    reca=p_rec(sig)
    if a==1
        reca=reca-1
    elseif a==2
        reca=reca-4
    elseif a==3
        reca=reca-10
    end
    return reca
end 

function r_exp(s,a,rec,S,A)
    suma=0.0
    for i in S
        suma+=p(i,s,a)*rec(i,s,a)
    end
    return suma 
end

function construir_r(rec,a,S,A)
    r=Float64[]
    for (i,s) in enumerate(S)
        append!(r,r_exp(s,a[i],rec,S,A))
    end    
    return r
end

construir_r (generic function with 1 method)

Ahora implementemos el método de iteración por política para el proceso con recompensa promedio de largo plazo.

In [6]:
using LinearAlgebra

In [7]:
function iteracion_largo_plazo(rec,S,A,d0p)
    d0=deepcopy(d0p)
    d1=deepcopy(d0p)
    seguir=true
    g=0.0
    h=zeros(length(S))
    while seguir
        println(d1)
        Pd=construir_matriz(d1,S)
        Q=I(length(S))-Pd
        Q[:,1].=1
        r=construir_r(rec,d1,S,A)
        h=Q\r
        g=copy(h[1])
        h[1]=0
        anew=deepcopy(d1)
        Pdh=Pd*h
        for (i,s) in enumerate(S)
            val=[r_exp(s,a,rec,S,A)+Pdh[i] for a in A[i]]
            anew[i]=A[i][argmax(val)]
        end
        d0=deepcopy(d1)
        d1=deepcopy(anew)
        if d1==d0
            seguir=false
        end
    end
    return g,h,d1
end

iteracion_largo_plazo (generic function with 1 method)

In [8]:
S=[1,2,3,4]
A=[[1,2,3],[1,2,3],[1,2,3],[1,2,3]]

4-element Vector{Vector{Int64}}:
 [1, 2, 3]
 [1, 2, 3]
 [1, 2, 3]
 [1, 2, 3]

In [9]:
g,h,polopt=iteracion_largo_plazo(r1,S,A,[1,1,1,1])

[1, 1, 1, 1]
[2, 2, 2, 3]


(4.620334620334621, [0.0, -1.8983268983269, -7.983268983268985, -8.06949806949807], [2, 2, 2, 3])

La política óptima que se obtuvo fue (2,2,2,3).

## Punto 3

Implementemos ahora la iteración por política descontada

In [10]:
function iteracion_descontada(λ,d0p,rec,S,A)
    d0=deepcopy(d0p)
    d1=deepcopy(d0p)
    seguir=true
    while seguir
        Pd=construir_matriz(d1,S)
        Q=I(length(S))-λ*Pd
        r=construir_r(rec,d1,S,A)
        v=Q\r
        pnew=deepcopy(d1)
        Pdv=Pd*v
        for (i,s) in enumerate(S)
            val=[r_exp(s,a,rec,S,A)+λ*Pdv[i] for a in A[i]]
            pnew[i]=A[i][argmax(val)]
        end
        d0=deepcopy(d1)
        d1=deepcopy(pnew)
        if d1==d0
            seguir=false
        end
    end
    return d1
end

iteracion_descontada (generic function with 1 method)

In [27]:
pol=iteracion_descontada(0.9,[1,1,1,1],r1,S,A)

4-element Vector{Int64}:
 2
 2
 2
 3

Vemos que con $\lambda=0.9$ obtenemos la misma política óptima estacionaria (2,2,2,3).

## Punto 4 

Ahora, para maximizar la recompensa promedio esperada a largo plazo sujeto a no estar más del 10% del tiempo en el estado 4 usamos programación lineal.

Para un problema sin restricciones el problema dual que hay resolver para obtener las medidas de ocupación es

\begin{equation}
\begin{split}
\text{max } &\sum_{s,a}r(s,a)x(s,a)\\
\sum_{a\in A_s}x(j,a)-&\sum_{s,a}p(j|s,a)x(s,a)=0, \quad j \in S\\
&\sum_{s,a}x(s,a)=1\\
&x(s,a)\geq 0 \quad s\in S \quad a\in A_s 
\end{split}
\end{equation}

En este caso, las variables duales $x(s,a)$ se pueden interpretar como la probabilidad limite de encontrarse en el estado $s$ y escoger la acción $a$ con la política aleatoria definida por

\begin{equation}
q_{d(s)}(a)=\frac{x(s,a)}{\sum_{a'}x(s,a')}
\end{equation}

Así, para modelar la restricción propuesta, se añade la restricción adicional

\begin{equation}
\sum_{a}x(4,a)\leq 0.1
\end{equation}

Ahora implementemoslo

In [46]:
using JuMP
using Clp

In [32]:
function r2(sig,s,a)
    reca=0
    if a==1
        reca=reca-1
    elseif a==2
        reca=reca-4
    elseif a==3
        reca=reca-10
    end
    return reca
end 

r2 (generic function with 1 method)

In [47]:
function linear_programming(S,A,rec)
    model = Model(Clp.Optimizer)
    N=length(S)
    @variable(model, x[s=1:N,a=1:length(A[s])])
    @constraint(model,c[j=1:N],sum(x[j,a] for a in 1:length(A[j]))-sum(p(S[j],S[s],A[s][a])*x[s,a] for s in 1:N for a in 1:length(A[s]))==0)
    @constraint(model,cnorm,sum(x[s,a] for s in 1:N for a in 1:length(A[s]))==1.0)
    @constraint(model,cpos[s=1:N,a=1:length(A[s])], x[s,a]>=0.0)
    @constraint(model,cconst, sum(x[4,a] for a in 1:length(A[4]))<=0.1)
    @objective(model, Max, sum(r_exp(S[s],A[s][a],rec,S,A)*x[s,a] for s in 1:N for a in 1:length(A[s])))
    println(model)
    optimize!(model)
    medidas=[]
    for i in 1:length(S)
        mes=[]
        for j in 1:length(A[i])
            append!(mes,value(x[i,j]))
        end
        append!(medidas,[mes])
    end
    return model,medidas
end

linear_programming (generic function with 1 method)

Y resolvamos el problema

In [48]:
model,medidas=linear_programming(S,A,r2);

Max -x[1,1] - 4 x[1,2] - 10 x[1,3] - x[2,1] - 3.9999999999999996 x[2,2] - 10 x[2,3] - x[3,1] - 3.9999999999999996 x[3,2] - 10 x[3,3] - x[4,1] - 4 x[4,2] - 10 x[4,3]
Subject to
 c[1] : 0.9 x[1,1] + 0.19999999999999996 x[1,2] + 0.050000000000000044 x[1,3] - 0.7 x[2,2] - 0.85 x[2,3] - 0.3 x[3,2] - 0.65 x[3,3] - 0.5 x[4,3] = 0.0
 c[2] : -0.6 x[1,1] - 0.2 x[1,2] - 0.05 x[1,3] + 0.9 x[2,1] + 0.8 x[2,2] + 0.9 x[2,3] - 0.4 x[3,2] - 0.2 x[3,3] - 0.6 x[4,2] - 0.5 x[4,3] = 0.0
 c[3] : -0.2 x[1,1] - 0.6 x[2,1] - 0.1 x[2,2] - 0.05 x[2,3] + 0.8 x[3,1] + 0.8 x[3,2] + 0.9 x[3,3] - 0.2 x[4,2] = 0.0
 c[4] : -0.1 x[1,1] - 0.3 x[2,1] - 0.8 x[3,1] - 0.1 x[3,2] - 0.05 x[3,3] + 0.8 x[4,2] + x[4,3] = 0.0
 cnorm : x[1,1] + x[1,2] + x[1,3] + x[2,1] + x[2,2] + x[2,3] + x[3,1] + x[3,2] + x[3,3] + x[4,1] + x[4,2] + x[4,3] = 1.0
 cpos[1,1] : x[1,1] ≥ 0.0
 cpos[1,2] : x[1,2] ≥ 0.0
 cpos[1,3] : x[1,3] ≥ 0.0
 cpos[2,1] : x[2,1] ≥ 0.0
 cpos[2,2] : x[2,2] ≥ 0.0
 cpos[2,3] : x[2,3] ≥ 0.0
 cpos[3,1] : x[3,1] ≥ 0.0
 cpos[3

La solución se obtiene con Cpl y es

In [50]:
medidas

4-element Vector{Any}:
 Any[0.2988479262672811, 0.0, 0.0]
 Any[0.09723502304147469, 0.2944700460829496, 0.0]
 Any[0.0, 0.20944700460829493, 0.0]
 Any[0.0, 0.1, 0.0]

que corresponde a una política markoviana aleatoria, que al estado 1 le asigna la acción 1 con probabilidad 1, al estado 2 le asigna la acción 1 con probabilidad aproximada de 0.25 y la acción 2 con probabilidad de 0.75, al estado tres le asigna la acción 2 con probabilidad 1 y al estado 4 le asigna la acción 2 con probabilidad 1.

## Conclusiones

- Al ser pequeño el tamaño del espacio de estados y pocas acciones, los algoritmos iterativos convergen muy rápido a la política óptima.
- Es más fácil imponer restricciones al problema en la formulación lineal usando la interpretación de las variables duales
- La política óptima para el problema restrringido no necesariamente será determinista, ya que la esquinas del polítopo ya no corresponden todas a estas políticas.